In [44]:
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup
import re
from tqdm import tqdm_notebook
from yaml import load, dump
from getbibli import *
import json
import multiprocessing as mp

In [45]:
def download(book):
    '''retireve the mp3 links from the url'''
    bookpage =  BeautifulSoup(simple_get(book.url), 'html.parser')
    files = bookpage.find_all(
        "a", class_="link-mp3-file") + bookpage.find_all("a", class_="link-roman-mp3-file")
    book.files = [f['href'] for f in files]
    return book

class Book:
    def __init__(self, author, title, category, url):
        self.author =  author
        self.title = title
        self.category =  category
        self.url = url
        self.files = []
        
    def isDownloaded(self):
        return len(self.files) > 0
        
    def toJson(self):
        return json.dumps(self, default=lambda o: o.__dict__, 
            sort_keys=True, indent=4)

    def __repr__(self):
        return str(self.__dict__)
        

In [46]:


title_matcher = re.compile("(.+) [-–] (.+)")
bibli = []
for page in tqdm_notebook(pages[:1]):
    raw_html = simple_get(page)
    html = BeautifulSoup(raw_html, 'html.parser')
    for p in html.select('ul'):
        if 'Catégorie' in p.select_one('li').text:
            break
    
    for li in tqdm_notebook(p.findChildren("li", recursive=False)):

        booklist = li.find('ul')
        category = booklist.previousSibling
        category = str(category).split(':')[1].strip() if 'Catégorie' in category else category.text
        print(category)
        c = 0
        for b in tqdm_notebook(booklist.findChildren("li", recursive=False)):
            link = b.select_one('a')
            m = title_matcher.match(link['title'])
            assert m is not None, f"title {link['title']} doesn't match"
            bibli.append(Book(
                author = m.group(1),
                title = m.group(2),
                category = category,
                url = link['href']
            ))
            
            


Arts


Aventure


Érotisme


Biographies


In [47]:
download(bibli[0])

{'author': "AXA, Zo (d')", 'title': "Par l'image", 'category': 'Arts', 'url': 'http://www.litteratureaudio.com/livre-audio-gratuit-mp3/axa-zo-d-par-limage.html', 'files': ['http://www.litteratureaudio.org/mp3/Zo_d_Axa_-_Endehors_Par_l_Image.mp3']}

In [53]:
%%time 
results=[]
for b in bibli[:10]:
    results.append(download(b))
print(results)

[{'author': "AXA, Zo (d')", 'title': "Par l'image", 'category': 'Arts', 'url': 'http://www.litteratureaudio.com/livre-audio-gratuit-mp3/axa-zo-d-par-limage.html', 'files': ['http://www.litteratureaudio.org/mp3/Zo_d_Axa_-_Endehors_Par_l_Image.mp3']}, {'author': 'BAUDELAIRE, Charles', 'title': 'Richard Wagner et Tannhäuser à Paris', 'category': 'Arts', 'url': 'http://www.litteratureaudio.com/livre-audio-gratuit-mp3/baudelaire-charles-richard-wagner-et-tannhauser-a-paris.html', 'files': ['http://www.litteratureaudio.org/mp3/Charles_Baudelaire_-_Richard_Wagner_et_Tannhauser_a_Paris_Chap1.mp3', 'http://www.litteratureaudio.org/mp3/Charles_Baudelaire_-_Richard_Wagner_et_Tannhauser_a_Paris_Chap1.mp3', 'http://www.litteratureaudio.org/mp3/Charles_Baudelaire_-_Richard_Wagner_et_Tannhauser_a_Paris_Chap2.mp3', 'http://www.litteratureaudio.org/mp3/Charles_Baudelaire_-_Richard_Wagner_et_Tannhauser_a_Paris_Chap3.mp3', 'http://www.litteratureaudio.org/mp3/Charles_Baudelaire_-_Richard_Wagner_et_Tannha

In [55]:
%%time 
pool = mp.Pool(processes=10)
results = pool.map(download, bibli[:10])
print(results)

[{'author': "AXA, Zo (d')", 'title': "Par l'image", 'category': 'Arts', 'url': 'http://www.litteratureaudio.com/livre-audio-gratuit-mp3/axa-zo-d-par-limage.html', 'files': ['http://www.litteratureaudio.org/mp3/Zo_d_Axa_-_Endehors_Par_l_Image.mp3']}, {'author': 'BAUDELAIRE, Charles', 'title': 'Richard Wagner et Tannhäuser à Paris', 'category': 'Arts', 'url': 'http://www.litteratureaudio.com/livre-audio-gratuit-mp3/baudelaire-charles-richard-wagner-et-tannhauser-a-paris.html', 'files': ['http://www.litteratureaudio.org/mp3/Charles_Baudelaire_-_Richard_Wagner_et_Tannhauser_a_Paris_Chap1.mp3', 'http://www.litteratureaudio.org/mp3/Charles_Baudelaire_-_Richard_Wagner_et_Tannhauser_a_Paris_Chap1.mp3', 'http://www.litteratureaudio.org/mp3/Charles_Baudelaire_-_Richard_Wagner_et_Tannhauser_a_Paris_Chap2.mp3', 'http://www.litteratureaudio.org/mp3/Charles_Baudelaire_-_Richard_Wagner_et_Tannhauser_a_Paris_Chap3.mp3', 'http://www.litteratureaudio.org/mp3/Charles_Baudelaire_-_Richard_Wagner_et_Tannha

In [15]:
json.dump(bibli,  open('bibli.json','w'))

In [31]:
dump({'Bibliothèques':{'Catégories': bibli},    
    "Reprendre la lecture en cours":None,
    "quel heure est-t-il?":None,"Quitter l'application":None},  open('bibli.yaml','w'),allow_unicode=True)

In [ ]:
results